In [141]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, spearmanr
from talib import RSI, BBANDS, MACD, ATR

<font size="5">Load Stock prices and Data

In [142]:
DATA_STORE = '../data/assets.h5'

In [143]:
START = '2013-01-01'
END = '2017-12-31'

MONTH = 21
YEAR = 12 * MONTH

In [144]:
sns.set_style('whitegrid')
idx = pd.IndexSlice
ohlcv = ['adj_open', 'adj_close', 'adj_low', 'adj_high', 'adj_volume']

In [145]:
with pd.HDFStore(DATA_STORE) as store:
    prices = (store['quandl/wiki/prices']
              .loc[idx[START:END, :], ohlcv]
              .rename(columns=lambda x: x.replace('adj_', ''))
              .assign(volume=lambda x: x.volume.div(1000))
              .swaplevel()
              .sort_index())

    stocks = (store['us_equities/stocks']
              .loc[:, ['marketcap', 'ipoyear', 'sector']])

In [146]:
prices

open      close        low       high    volume
ticker date                                                            
A      2013-01-02  29.054711  28.854904  28.351942  29.054711  6287.700
       2013-01-03  28.896244  28.958253  28.648276  29.034042  4114.300
       2013-01-04  29.034042  29.530115  28.841124  29.592124  4601.500
       2013-01-07  29.350977  29.316528  29.199399  29.488775  2567.600
       2013-01-08  29.261408  29.082271  29.006482  29.474995  2787.500
...                      ...        ...        ...        ...       ...
ZUMZ   2017-12-22  21.550000  21.450000  21.300000  21.700000   209.585
       2017-12-26  21.450000  21.850000  21.450000  22.200000   213.951
       2017-12-27  21.800000  21.150000  20.950000  22.250000   227.325
       2017-12-28  21.200000  21.200000  21.050000  21.450000   162.070
       2017-12-29  21.250000  20.825000  20.800000  21.450000   222.040

[3619403 rows x 5 columns]

In [147]:
stocks

,marketcap,ipoyear,sector
ticker,,,
PIH,4.309000e+07,2014.0,Finance
PIHPP,NaN,NaN,Finance
TURN,7.033000e+07,NaN,Finance
FLWS,8.333900e+08,1999.0,Consumer Services
FCCY,1.781400e+08,NaN,Finance
...,...,...,...
ZOES,1.974000e+08,2014.0,Consumer Services
ZTS,4.165000e+10,2013.0,Health Care
ZTO,1.434000e+10,2016.0,Transportation


<font size="5"> Clean the Data and delete stock with few observations

In [148]:
min_year = 2 * YEAR

observation = prices.groupby(level='ticker').size()

keep = observation[observation > min_year].index

In [149]:
prices = prices.loc[idx[keep], :]

In [150]:
stocks = stocks[~stocks.index.duplicated() & stocks.sector.notnull()]
stocks.index.name = 'ticker'

In [151]:
stocks

,marketcap,ipoyear,sector
ticker,,,
PIH,4.309000e+07,2014.0,Finance
PIHPP,NaN,NaN,Finance
TURN,7.033000e+07,NaN,Finance
FLWS,8.333900e+08,1999.0,Consumer Services
FCCY,1.781400e+08,NaN,Finance
...,...,...,...
ZOES,1.974000e+08,2014.0,Consumer Services
ZTS,4.165000e+10,2013.0,Health Care
ZTO,1.434000e+10,2016.0,Transportation


In [152]:
shared_stocks = prices.index.get_level_values('ticker').unique().intersection(stocks.index)
stocks = stocks.loc[shared_stocks, :]
prices = prices.loc[idx[shared_stocks], :]

In [153]:
stocks

,marketcap,ipoyear,sector
ticker,,,
A,1.960000e+10,1999.0,Capital Goods
AAL,1.767000e+10,NaN,Transportation
AAMC,1.104900e+08,NaN,Finance
AAN,3.200000e+09,NaN,Technology
AAOI,9.000500e+08,2013.0,Technology
...,...,...,...
ZIXI,2.944800e+08,NaN,Technology
ZNGA,3.550000e+09,2011.0,Technology
ZOES,1.974000e+08,2014.0,Consumer Services


In [154]:
prices.groupby(level="ticker").count()

,open,close,low,high,volume
ticker,,,,,
A,1258,1258,1258,1258,1258
AAL,1258,1258,1258,1258,1258
AAMC,1258,1258,1258,1258,1258
AAN,1258,1258,1258,1258,1258
AAOI,1073,1073,1073,1073,1073
...,...,...,...,...,...
ZIXI,1258,1258,1258,1258,1258
ZNGA,1258,1258,1258,1258,1258
ZOES,937,937,937,937,937


In [155]:
stocks.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2348 entries, A to ZUMZ
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   marketcap  2345 non-null   float64
 1   ipoyear    1026 non-null   float64
 2   sector     2348 non-null   object 
dtypes: float64(2), object(1)
memory usage: 137.9+ KB


In [156]:
prices.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2904233 entries, ('A', Timestamp('2013-01-02 00:00:00')) to ('ZUMZ', Timestamp('2017-12-29 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    2904233 non-null  float64
 1   close   2904233 non-null  float64
 2   low     2904233 non-null  float64
 3   high    2904233 non-null  float64
 4   volume  2904233 non-null  float64
dtypes: float64(5)
memory usage: 122.6+ MB


<font size="5">  Save Data

In [157]:
with pd.HDFStore('tmp.h5') as store:
     store.put('prices', prices)
     store.put('stocks', stocks)

In [158]:
prices['dollar_volume'] = prices[['close', 'volume']].prod(axis=1)

In [159]:
prices

open      close        low       high    volume  \
ticker date                                                               
A      2013-01-02  29.054711  28.854904  28.351942  29.054711  6287.700   
       2013-01-03  28.896244  28.958253  28.648276  29.034042  4114.300   
       2013-01-04  29.034042  29.530115  28.841124  29.592124  4601.500   
       2013-01-07  29.350977  29.316528  29.199399  29.488775  2567.600   
       2013-01-08  29.261408  29.082271  29.006482  29.474995  2787.500   
...                      ...        ...        ...        ...       ...   
ZUMZ   2017-12-22  21.550000  21.450000  21.300000  21.700000   209.585   
       2017-12-26  21.450000  21.850000  21.450000  22.200000   213.951   
       2017-12-27  21.800000  21.150000  20.950000  22.250000   227.325   
       2017-12-28  21.200000  21.200000  21.050000  21.450000   162.070   
       2017-12-29  21.250000  20.825000  20.800000  21.450000   222.040   

                   dollar_volume  
ticker date                       
A      2013-01-02  181430.981611  
       2013-01-03  119142.939438  
       2013-01-04  135882.822089  
       2013-01-07   75273.116328  
       2013-01-08   81066.830411  
...                          ...  
ZUMZ   2017-12-22    4495.598250  
       2017-12-26    4674.829350  
       2017-12-27    4807.923750  
       2017-12-28    3435.884000  
       2017-12-29    4623.983000  

[2904233 rows x 6 columns]

In [160]:
prices["dollar_volume_1Month"] = prices["dollar_volume"].groupby('ticker').rolling(window = 21, level = "date").mean().values
prices

open      close        low       high    volume  \
ticker date                                                               
A      2013-01-02  29.054711  28.854904  28.351942  29.054711  6287.700   
       2013-01-03  28.896244  28.958253  28.648276  29.034042  4114.300   
       2013-01-04  29.034042  29.530115  28.841124  29.592124  4601.500   
       2013-01-07  29.350977  29.316528  29.199399  29.488775  2567.600   
       2013-01-08  29.261408  29.082271  29.006482  29.474995  2787.500   
...                      ...        ...        ...        ...       ...   
ZUMZ   2017-12-22  21.550000  21.450000  21.300000  21.700000   209.585   
       2017-12-26  21.450000  21.850000  21.450000  22.200000   213.951   
       2017-12-27  21.800000  21.150000  20.950000  22.250000   227.325   
       2017-12-28  21.200000  21.200000  21.050000  21.450000   162.070   
       2017-12-29  21.250000  20.825000  20.800000  21.450000   222.040   

                   dollar_volume  dollar_volume_1Month  
ticker date                                             
A      2013-01-02  181430.981611                   NaN  
       2013-01-03  119142.939438                   NaN  
       2013-01-04  135882.822089                   NaN  
       2013-01-07   75273.116328                   NaN  
       2013-01-08   81066.830411                   NaN  
...                          ...                   ...  
ZUMZ   2017-12-22    4495.598250          11930.001226  
       2017-12-26    4674.829350          11963.373076  
       2017-12-27    4807.923750          11786.938512  
       2017-12-28    3435.884000          11290.208650  
       2017-12-29    4623.983000          10790.612631  

[2904233 rows x 7 columns]

In [161]:
prices['dollar_volume_rank'] = (prices['dollar_volume_1Month'].groupby('date')
                             .rank(ascending=False))
prices

open      close        low       high    volume  \
ticker date                                                               
A      2013-01-02  29.054711  28.854904  28.351942  29.054711  6287.700   
       2013-01-03  28.896244  28.958253  28.648276  29.034042  4114.300   
       2013-01-04  29.034042  29.530115  28.841124  29.592124  4601.500   
       2013-01-07  29.350977  29.316528  29.199399  29.488775  2567.600   
       2013-01-08  29.261408  29.082271  29.006482  29.474995  2787.500   
...                      ...        ...        ...        ...       ...   
ZUMZ   2017-12-22  21.550000  21.450000  21.300000  21.700000   209.585   
       2017-12-26  21.450000  21.850000  21.450000  22.200000   213.951   
       2017-12-27  21.800000  21.150000  20.950000  22.250000   227.325   
       2017-12-28  21.200000  21.200000  21.050000  21.450000   162.070   
       2017-12-29  21.250000  20.825000  20.800000  21.450000   222.040   

                   dollar_volume  dollar_volume_1Month  dollar_volume_rank  
ticker date                                                                 
A      2013-01-02  181430.981611                   NaN                 NaN  
       2013-01-03  119142.939438                   NaN                 NaN  
       2013-01-04  135882.822089                   NaN                 NaN  
       2013-01-07   75273.116328                   NaN                 NaN  
       2013-01-08   81066.830411                   NaN                 NaN  
...                          ...                   ...                 ...  
ZUMZ   2017-12-22    4495.598250          11930.001226              1288.0  
       2017-12-26    4674.829350          11963.373076              1291.0  
       2017-12-27    4807.923750          11786.938512              1293.0  
       2017-12-28    3435.884000          11290.208650              1300.0  
       2017-12-29    4623.983000          10790.612631              1312.0  

[2904233 rows x 8 columns]

<font size="5">  ADD Stock Factors

RSI

In [162]:
prices['rsi'] = prices['close'].groupby(level='ticker').apply(RSI)
prices

open      close        low       high    volume  \
ticker date                                                               
A      2013-01-02  29.054711  28.854904  28.351942  29.054711  6287.700   
       2013-01-03  28.896244  28.958253  28.648276  29.034042  4114.300   
       2013-01-04  29.034042  29.530115  28.841124  29.592124  4601.500   
       2013-01-07  29.350977  29.316528  29.199399  29.488775  2567.600   
       2013-01-08  29.261408  29.082271  29.006482  29.474995  2787.500   
...                      ...        ...        ...        ...       ...   
ZUMZ   2017-12-22  21.550000  21.450000  21.300000  21.700000   209.585   
       2017-12-26  21.450000  21.850000  21.450000  22.200000   213.951   
       2017-12-27  21.800000  21.150000  20.950000  22.250000   227.325   
       2017-12-28  21.200000  21.200000  21.050000  21.450000   162.070   
       2017-12-29  21.250000  20.825000  20.800000  21.450000   222.040   

                   dollar_volume  dollar_volume_1Month  dollar_volume_rank  \
ticker date                                                                  
A      2013-01-02  181430.981611                   NaN                 NaN   
       2013-01-03  119142.939438                   NaN                 NaN   
       2013-01-04  135882.822089                   NaN                 NaN   
       2013-01-07   75273.116328                   NaN                 NaN   
       2013-01-08   81066.830411                   NaN                 NaN   
...                          ...                   ...                 ...   
ZUMZ   2017-12-22    4495.598250          11930.001226              1288.0   
       2017-12-26    4674.829350          11963.373076              1291.0   
       2017-12-27    4807.923750          11786.938512              1293.0   
       2017-12-28    3435.884000          11290.208650              1300.0   
       2017-12-29    4623.983000          10790.612631              1312.0   

                         rsi  
ticker date                   
A      2013-01-02        NaN  
       2013-01-03        NaN  
       2013-01-04        NaN  
       2013-01-07        NaN  
       2013-01-08        NaN  
...                      ...  
ZUMZ   2017-12-22  61.077094  
       2017-12-26  63.799097  
       2017-12-27  56.369727  
       2017-12-28  56.757082  
       2017-12-29  52.959469  

[2904233 rows x 9 columns]

bollinger band

In [163]:
high, mid, low = BBANDS(prices['close'], timeperiod=20)
prices["boling_high"] = high
prices["boling_low"] = low

In [164]:
prices['boling_high'] = prices['boling_high'].sub(prices['close']).div(prices['boling_high']).apply(np.log1p)
prices['boling_low'] = prices['close'].sub(prices['boling_low']).div(prices['close']).apply(np.log1p)

In [165]:
prices

open      close        low       high    volume  \
ticker date                                                               
A      2013-01-02  29.054711  28.854904  28.351942  29.054711  6287.700   
       2013-01-03  28.896244  28.958253  28.648276  29.034042  4114.300   
       2013-01-04  29.034042  29.530115  28.841124  29.592124  4601.500   
       2013-01-07  29.350977  29.316528  29.199399  29.488775  2567.600   
       2013-01-08  29.261408  29.082271  29.006482  29.474995  2787.500   
...                      ...        ...        ...        ...       ...   
ZUMZ   2017-12-22  21.550000  21.450000  21.300000  21.700000   209.585   
       2017-12-26  21.450000  21.850000  21.450000  22.200000   213.951   
       2017-12-27  21.800000  21.150000  20.950000  22.250000   227.325   
       2017-12-28  21.200000  21.200000  21.050000  21.450000   162.070   
       2017-12-29  21.250000  20.825000  20.800000  21.450000   222.040   

                   dollar_volume  dollar_volume_1Month  dollar_volume_rank  \
ticker date                                                                  
A      2013-01-02  181430.981611                   NaN                 NaN   
       2013-01-03  119142.939438                   NaN                 NaN   
       2013-01-04  135882.822089                   NaN                 NaN   
       2013-01-07   75273.116328                   NaN                 NaN   
       2013-01-08   81066.830411                   NaN                 NaN   
...                          ...                   ...                 ...   
ZUMZ   2017-12-22    4495.598250          11930.001226              1288.0   
       2017-12-26    4674.829350          11963.373076              1291.0   
       2017-12-27    4807.923750          11786.938512              1293.0   
       2017-12-28    3435.884000          11290.208650              1300.0   
       2017-12-29    4623.983000          10790.612631              1312.0   

                         rsi  boling_high  boling_low  
ticker date                                            
A      2013-01-02        NaN          NaN         NaN  
       2013-01-03        NaN          NaN         NaN  
       2013-01-04        NaN          NaN         NaN  
       2013-01-07        NaN          NaN         NaN  
       2013-01-08        NaN          NaN         NaN  
...                      ...          ...         ...  
ZUMZ   2017-12-22  61.077094     0.028426    0.086877  
       2017-12-26  63.799097     0.010058    0.090436  
       2017-12-27  56.369727     0.040577    0.057586  
       2017-12-28  56.757082     0.039229    0.059004  
       2017-12-29  52.959469     0.050887    0.042288  

[2904233 rows x 11 columns]

Average True Range

In [166]:
def compute_ATR(stock_data):
    data = ATR(stock_data.high, stock_data.low, 
             stock_data.close, timeperiod=14)
    return data.sub(data.mean()).div(data.std())


prices['ATR'] = (prices.groupby('ticker', group_keys=False)
                 .apply(compute_ATR))

prices

open      close        low       high    volume  \
ticker date                                                               
A      2013-01-02  29.054711  28.854904  28.351942  29.054711  6287.700   
       2013-01-03  28.896244  28.958253  28.648276  29.034042  4114.300   
       2013-01-04  29.034042  29.530115  28.841124  29.592124  4601.500   
       2013-01-07  29.350977  29.316528  29.199399  29.488775  2567.600   
       2013-01-08  29.261408  29.082271  29.006482  29.474995  2787.500   
...                      ...        ...        ...        ...       ...   
ZUMZ   2017-12-22  21.550000  21.450000  21.300000  21.700000   209.585   
       2017-12-26  21.450000  21.850000  21.450000  22.200000   213.951   
       2017-12-27  21.800000  21.150000  20.950000  22.250000   227.325   
       2017-12-28  21.200000  21.200000  21.050000  21.450000   162.070   
       2017-12-29  21.250000  20.825000  20.800000  21.450000   222.040   

                   dollar_volume  dollar_volume_1Month  dollar_volume_rank  \
ticker date                                                                  
A      2013-01-02  181430.981611                   NaN                 NaN   
       2013-01-03  119142.939438                   NaN                 NaN   
       2013-01-04  135882.822089                   NaN                 NaN   
       2013-01-07   75273.116328                   NaN                 NaN   
       2013-01-08   81066.830411                   NaN                 NaN   
...                          ...                   ...                 ...   
ZUMZ   2017-12-22    4495.598250          11930.001226              1288.0   
       2017-12-26    4674.829350          11963.373076              1291.0   
       2017-12-27    4807.923750          11786.938512              1293.0   
       2017-12-28    3435.884000          11290.208650              1300.0   
       2017-12-29    4623.983000          10790.612631              1312.0   

                         rsi  boling_high  boling_low       ATR  
ticker date                                                      
A      2013-01-02        NaN          NaN         NaN       NaN  
       2013-01-03        NaN          NaN         NaN       NaN  
       2013-01-04        NaN          NaN         NaN       NaN  
       2013-01-07        NaN          NaN         NaN       NaN  
       2013-01-08        NaN          NaN         NaN       NaN  
...                      ...          ...         ...       ...  
ZUMZ   2017-12-22  61.077094     0.028426    0.086877 -0.419131  
       2017-12-26  63.799097     0.010058    0.090436 -0.450751  
       2017-12-27  56.369727     0.040577    0.057586 -0.282948  
       2017-12-28  56.757082     0.039229    0.059004 -0.449763  
       2017-12-29  52.959469     0.050887    0.042288 -0.515043  

[2904233 rows x 12 columns]

MACD

In [167]:
def compute_macd(close):
    macd = MACD(close)[0]
    return (macd - np.mean(macd))/np.std(macd)

prices['MACD'] = (prices
                  .groupby('ticker', group_keys=False)
                  .close
                  .apply(compute_macd))

Lagged Return

In [177]:
lags = [1, 5, 10, 21, 42, 63]
returns = prices.groupby(level='ticker').close.pct_change()
percentiles=[.0001, .001, .01]
percentiles += [1-p for p in percentiles]

q = 0.0001

for lag in lags:
    prices[f'return_{lag}d'] = (prices.groupby(level='ticker').close
                                .pct_change(lag)
                                .pipe(lambda x: x.clip(lower=x.quantile(q),
                                                       upper=x.quantile(1 - q)))
                                .add(1)
                                .pow(1 / lag)
                                .sub(1)
                                )


In [169]:
for t in [1, 2, 3, 4, 5]:
    for lag in [1, 5, 10, 21]:
        prices[f'return_{lag}d_lag{t}'] = (prices.groupby(level='ticker')
                                           [f'return_{lag}d'].shift(t * lag))

Forward Return

In [170]:
for t in [1, 5, 10, 21]:
    prices[f'target_{t}d'] = prices.groupby(level='ticker')[f'return_{t}d'].shift(-t)

In [171]:
prices = prices.join(stocks[['sector']])
prices['year'] = prices.index.get_level_values('date').year
prices['month'] = prices.index.get_level_values('date').month

In [172]:
prices.assign(sector=pd.factorize(prices.sector, sort=True)[0]).to_hdf('data.h5', 'model_data/no_dummies')

In [173]:
prices = pd.get_dummies(prices,
                        columns=['year', 'month', 'sector'],
                        prefix=['year', 'month', ''],
                        prefix_sep=['_', '_', ''],
                        drop_first=True)

In [174]:
prices.to_hdf('data.h5', 'model_data')